Notice! Memoization has been implemented directly into the chess engine notebooks and not via this one. This one only contains the theory behind Memoization.

In [14]:
function memoize(f)
    global gCache
    function f_memoized(b, s, d)    # TODO: replace (b, s, d) with args... to make function generic
        if (b, s, d) in keys(gCache)
            return gCache[args]
        end
        result = f(b, s, d)
        merge!(gCache, Dict((b, s, d) => result))
        return result
    end
    return f_memoized
end

memoize (generic function with 1 method)

In [1]:
using Chess

using NBInclude

In [2]:
@nbinclude("ZobristHashing.ipynb")

zobrist_hash (generic function with 2 methods)

# Memoization

This notebook contains alle functions needed for memoizing function calls that we have already calculated. This will greatly reduce the runtime of the program.  

Declare global Variable gCache

In [6]:
gCache = Dict()

Dict{Any, Any}()

The `evaluate` function adds memoization to the `Max` and `Min` function.

It takes in 4 arguments and 2 optional arguments.
1. `State` is the current state represented by a `Board`
1. `f` takes in either the function `alphaBetaMax` and `alphaBetaMin`
1. `score` is the static centipawn evaluation of the `state`
1. `depth` is the number of halfmoves the engine should analyze before terminating
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process

The function `evaluate` evaluates the same result as the function `f`. Additionally it saves calculated results in the `gCache`. And uses any entries in the `gCache` if the same function has already been called.

In [ ]:
function evaluate(State, f, score, hash, depth, alpha = -Inf, beta = Inf)
    global gCache
    if hash in keys(gCache) 
        flag, v, d = gCache[hash]
        if d >= depth   # check if stored depth to this position is already higher than required
            if flag == "="
                return v
            elseif flag == "<="
                if v <= alpha
                    return v
                else
                    beta = min(beta, v)
                    w = f(State, score, hash, depth, alpha, beta)
                    store_cache(hash, depth, alpha, beta, w)
                    return w
                end
            elseif flag == ">="
                if beta <= v
                    return v
                else
                    alpha = max(alpha, v)
                    w = f(State, score, hash, depth, v, beta)
                    store_cache(hash, depth, v, beta, w)
                    return w
                end
            end
        end
    end
    # no value stored in gCache for State or depth of stored State has less depth than required
    v = f(State, score, hash, depth, alpha, beta)
    store_cache(hash, depth, alpha, beta, v)
    return v
end

evaluate (generic function with 3 methods)

The `store_cache` is a helping function that stores values into the `gCache`. 

It takes 5 arguments:
1. `State` is the current `state` of type `Board`
1. `depth` is the number of halfmoves the engine should analyze before terminating
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process
1. `v` is the value that got calculated by the `evaluate` function that needs to be stored


In [ ]:
function store_cache(hash, depth, alpha, beta, v)
    global gCache
    if v <= alpha
        gCache[hash] = ("<=", v, depth)
    elseif v < beta
        gCache[hash] = ("=", v, depth)
    else # beta <= v
        gCache[hash] = (">=", v, depth)
    end
end

store_cache (generic function with 1 method)

In [8]:
function updateBoardData(board, score, hash, move)
    next_score = evaluate_move(board, move, score)
    next_hash = zobrist_hash(board, hash, move)
    return next_score, next_hash
end

updateBoardData (generic function with 2 methods)